In [ ]:
import os
os.chdir("..")

In [ ]:
import numpy as np
import spreg
import libpysal
import bambi as bmb
import arviz as az
import pandas as pd
import geopandas as gpd
import numpy
from pysal.lib import weights
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from pysal.lib import cg as geometry

from spreg import dgp_lag

from src.data.data_reg import SpatialReg

sr = SpatialReg()


In [ ]:
df = sr.spatial_simulation(n=10,time=5,rho=.8,simulations=10)
df

In [ ]:
sr.results

In [ ]:
import polars as pl
(df.select((pl.col("bayes_w")-.8)**2).sum()/10).item()

In [ ]:
df.select((pl.col("freq_w")-.8)**2).sum()/10

In [ ]:
gdf = sr.spatial_panel(n=10,time=4,rho=.8)
gdf

In [ ]:
wr = weights.contiguity.Rook.from_dataframe(gdf[gdf["time"] == 0])
wr.transform = "r"
y_d = gdf["y_d"].values.reshape(-1,1)
xb = gdf["X1"].values.reshape(-1,1)
fe_lag = spreg.Panel_FE_Lag(y=y_d, x=xb, w=wr)
print(fe_lag.betas[1][0])

In [ ]:
print(fe_lag.summary)

In [ ]:
import io
import contextlib
df = gdf.drop("geometry", axis=1)
model = bmb.Model(
    "y_d ~ 1 + X1 + w_d",
    df, dropna=True
)
with contextlib.redirect_stdout(io.StringIO()):
    results = model.fit()

In [ ]:
# betas = fe_lag.betas  # Replace with the actual betas
# betas_mean = betas.mean(axis=0)
az_summary = az.summary(results, hdi_prob=0.95)  # You can adjust hdi_prob if needed
means = az_summary['mean']
# Create a dictionary to store both results
# Combine both into a DataFrame
means.iloc[0]

In [ ]:
az.summary(results)

In [ ]:
df = gdf.drop("geometry", axis=1)
model = bmb.Model(
    "y_d ~ 0 + X1 + w_d",
    df, dropna=True
)
results = model.fit()

In [ ]:
az.plot_trace(
    results,
    compact=True,
)